In [2]:
import pandas as pd
import json

# 读取JSON文件
### !!!这里需要改数据加载地址
with open('/mnt/workspace/pangtianqi/medical_kb_chatbot/data/new_sample_drugs_0811.json', 'r', encoding='utf-8') as file:
    med_details = json.load(file)

In [3]:
from sentence_transformers import SentenceTransformer

# 加载模型到特定的CUDA设备
### !!!这里需要改模型地址
encoder = SentenceTransformer('/mnt/workspace/pangtianqi/medical_kb_chatbot/moka-ai/m3e-base', device='cuda:0')
embedding_size = encoder.get_sentence_embedding_dimension()

print(f"Embedding size: {embedding_size}")


/home/pai/envs/kb_chat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-13 09:02:28,258] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Embedding size: 768


In [4]:
## 标签 label，药品库 QA库等
## 大类 category
## 二级分类 sub_category
## 属性 attribute
## 展示来源 title 

In [5]:
# 写入向量数据库
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus
client = QdrantClient(host="121.5.254.174", port=49344, timeout=50000)

my_collection = "drug_test"

first_collection = client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=embedding_size, distance=models.Distance.COSINE)
)

In [6]:
# 药
import time
start_time = time.time()
drug_points = []
tmp_drug_points_dict = {}
tmp_drug_points_dict['ids'],tmp_drug_points_dict['text'],tmp_drug_points_dict['payload']=[],[],[]
j=0
# 药物
for drug_item in med_details:
    common_name = drug_item['通用名称']
    comd_name = drug_item['商品名称']
    choose_cols = ['通用名称','商品名称','适应症', '禁忌症', '注意事项', '药物相互作用', '主要成分','用法用量','儿童禁忌', '孕妇禁忌', '老年人禁忌']
    emb_cols = ['适应症', '禁忌症', '注意事项', '药物相互作用', '主要成分','用法用量','儿童禁忌', '孕妇禁忌', '老年人禁忌']
    for col in emb_cols:
        if col not in drug_item or common_name is None or drug_item[col] is None or drug_item[col]=='' or drug_item[col] =='-':
            continue
        text = common_name+' '+comd_name+' '+col+' '+drug_item[col] if comd_name is not None or comd_name != '-' else common_name+' '+col+' '+drug_item[col]
        
        meta_data = {key: drug_item[key] for key in choose_cols if drug_item[key] not in [None, "-",""]}
        selected_data = {}
        selected_data['meta_data'] = meta_data
        selected_data['embed_text'] = text
        selected_data['label'] = '药品库'
        selected_data['category'] = drug_item['药品分类']
        selected_data['title'] = common_name
        selected_data['attribute'] = col
        tmp_drug_points_dict['ids'].append(j)
        tmp_drug_points_dict['text'].append(text)
        tmp_drug_points_dict['payload'].append(selected_data)
        j+=1
        
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time spent: {elapsed_time:.5f} seconds")
#print("tmp_drug_points_dict",tmp_drug_points_dict['payload'])

Time spent: 0.02571 seconds


In [13]:
# from chatglm1 import *
# from chatglm2 import *
# #加载模型
# # config = ChatGLMConfig()
# # model = MyMLP(config)
# model =  MyGLU(hidden_size = 768)
# device = model.device
# model.load_state_dict(torch.load("model/best_chatglm1.ph"))
# #model.load_state_dict(torch.load("model/best_chatglm1.ph"))
# model = model.to(device)
# # print(model)
# # encoder = model.embed_model
# dim =768

In [8]:
# 分批embedding与插入
import time
start_time = time.time()
batch_size = 500
ids = tmp_drug_points_dict['ids']
payloads = tmp_drug_points_dict['payload']
text = tmp_drug_points_dict['text']


for i in range(0, len(ids), batch_size):
    
    batch_ids = ids[i:i + batch_size]
    batch_payloads = payloads[i:i + batch_size]
    batch_text = text[i:i + batch_size]
    embed_start_time = time.time()
    # 数据嵌入
    batch_vectors = model(batch_text).tolist()
    embed_end_time = time.time()
    # print(f"{i} Embedding Time spent: {embed_end_time - embed_start_time:.5f} seconds")
    qrant_start_time = time.time()
    # 数据插入
    client.upsert(
        collection_name=my_collection,
        points=models.Batch(
            ids=batch_ids,
            payloads=batch_payloads,
            vectors=batch_vectors
        ),
    )
    qrant_end_time = time.time()
    # print(f"{i} qrant insert Time spent: {qrant_end_time - qrant_start_time:.5f} seconds")
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time spent: {elapsed_time:.5f} seconds")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_243471/3730063359.py:17 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_243471/3730063359.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [9]:
# 分批embedding与插入
import time
start_time = time.time()
batch_size = 500
ids = tmp_drug_points_dict['ids']
payloads = tmp_drug_points_dict['payload']
text = tmp_drug_points_dict['text']


for i in range(0, len(ids), batch_size):
    
    batch_ids = ids[i:i + batch_size]
    batch_payloads = payloads[i:i + batch_size]
    batch_text = text[i:i + batch_size]
    embed_start_time = time.time()
    # 数据嵌入
    batch_vectors = encoder.encode(batch_text).tolist()
    embed_end_time = time.time()
    # print(f"{i} Embedding Time spent: {embed_end_time - embed_start_time:.5f} seconds")
    qrant_start_time = time.time()
    # 数据插入
    client.upsert(
        collection_name=my_collection,
        points=models.Batch(
            ids=batch_ids,
            payloads=batch_payloads,
            vectors=batch_vectors
        ),
    )
    qrant_end_time = time.time()
    # print(f"{i} qrant insert Time spent: {qrant_end_time - qrant_start_time:.5f} seconds")
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time spent: {elapsed_time:.5f} seconds")

Time spent: 28.71628 seconds


In [10]:
collection_info = client.get_collection(collection_name=my_collection)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 6182),
 ('indexed_vectors_count', 0),
 ('points_count', 6182),
 ('segments_count', 8),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)),
 ('payload_schema', {})]

## 查询

In [11]:
import time
start_time = time.time()
query = '我儿子最近经常咳嗽，喉咙里有痰，吃什么药好点？'
query_embed = model([query])[0].tolist()
res = client.search(
    collection_name=my_collection,
    query_vector=query_embed,
    limit=4
)
# print("res",res)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time spent: {elapsed_time:.2f} seconds")

res

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_243471/3384377626.py:4 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_243471/3384377626.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [12]:
import time
start_time = time.time()
query = '我有轻微抑郁症应该怎么办？'
query_embed = encoder.encode([query])[0]
res = client.search(
    collection_name=my_collection,
    query_vector=query_embed,
    limit=4
)
# print("res",res)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time spent: {elapsed_time:.2f} seconds")
res

Time spent: 0.13 seconds


[ScoredPoint(id=5290, version=10, score=0.8630241, payload={'attribute': '适应症', 'category': '化学药品', 'embed_text': '盐酸米那普仑片 - 适应症 适用于抑郁症患者。', 'label': '药品库', 'meta_data': {'主要成分': '本品主要成份为盐酸米那普仑，其化学名称：为（±)-顺式-2-(氨甲基)-N，N-二乙基-1-苯基环丙烷基甲酰胺盐酸盐。化学结构式：分子式：C15H22N2O·HCl分子量：282.81', '适应症': '适用于抑郁症患者。', '通用名称': '盐酸米那普仑片'}, 'title': '盐酸米那普仑片'}, vector=None),
 ScoredPoint(id=530, version=1, score=0.84938085, payload={'attribute': '适应症', 'category': '化学药品', 'embed_text': '氢溴酸西酞普兰 - 适应症 各种类型的抑郁症。', 'label': '药品库', 'meta_data': {'注意事项': '1.与单胺氧化酶抑制剂(MAOI’s)的相互作用：在同时使用5-羟色胺再摄取抑制剂(SSRI)和单胺氧化酶抑制剂(MAOI’s)的患者，曾报道发生严重或致命的反应，如体温过高、僵直、肌阵脔、重要生命体征的自主不稳定快速波动，包括精神极度兴奋所致的精神狂乱谵语或昏迷等意识改变;在近期停止使用SSRI并开始使用MAOI’s的患者也有发生上述反应的同样报道;有些病例表现出类似精神抑制药所引起的恶性综合症;此外，有限的动物试验资料显示联合使用SSRI和MAOI’s，两者的协同作用会升高血压，并且引起行为激动;所以建议SSRI和MAOI’s不可联合使用，或者停止服用其中任何一种药物至少14天后才可以服用另外一种药物;\n2.告诫：因本品解除抑制的作用可先于抗抑郁作用，所以，病人在出现明显抑郁缓解之前仍可能持续存在自杀的可能性;如病人进入躁狂期，应停用本品，并给予精神抑制药作适当治疗;\n3.西酞普兰可能会引发癫痫的发作，因此，有癫痫病史的患者慎用;\n4.西酞普兰可能会引发躁狂的发作，因此，有躁狂病史的患者慎用;\n5.严重肾功能障碍患者